In [ ]:
#!wget https://github.com/nvs543/AIML_B11_NLP/blob/c1666ca7a183eece7073c4c7e0946b8f46bfe1ed/bbc-fulltext.zip
#!wget https://github.com/nvs543/AIML_B11_NLP/blob/796491820603626de2fcfec6ae69fef3937a1866/bbc.zip
!wget /content/drive/MyDrive/Colab Notebooks/bbc.zip

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from urllib import request
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import os
import glob
import string

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
bbc_path = "/content/drive/MyDrive/Colab Notebooks/bbc/"

In [43]:
def readFiles(path,clsId):
  fileNames = []
  fNames = []
  articles = []
  clsIds = []
  #print(glob.glob(path+"*.txt"))
  filenames=os.listdir(path)
  
  for ef in filenames:
    f = open(path+ef).readlines()
    articles.append(" ".join(f))
    clsIds.append(clsId)
    fNames.append(ef)
  
  return fNames,articles,clsIds
  

In [44]:
classNames = ["business","entertainment","politics","sport","tech"]
allArticles = []
allClsIds = []
allFileNames = []
for idx in range(len(classNames)):
  p = bbc_path+classNames[idx]+"/"
  fn,ad,cid = readFiles(p,idx)
  allArticles = allArticles + ad
  allClsIds = allClsIds + cid
  allFileNames = allFileNames + fn
 
print(len(allArticles),len(allClsIds),len(allFileNames))

2225 2225 2225


In [45]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [46]:
for i in range(len(allArticles)):
  allArticles[i] = allArticles[i].lower() #convert to lower case
  allArticles[i] = re.sub('<.*?>',' ',allArticles[i]) # remove html tags
  allArticles[i] = re.sub('[^a-zA-Z]',' ',allArticles[i]) #remove non-alphabet
  allArticles[i] = re.sub(r'\s+[a-zA-Z]\s+',' ',allArticles[i]) #remove single character
  allArticles[i] = re.sub(r'\s+', ' ',allArticles[i]) #remvoe multiple spaces
  allArticles[i] = REPLACE_BY_SPACE_RE.sub(' ',allArticles[i]) # replace REPLACE_BY_SPACE_RE symbols by space in text
  allArticles[i] = BAD_SYMBOLS_RE.sub('',allArticles[i]) #remove symbols that are in BAD_SYMBOLS from text
  allArticles[i] = allArticles[i].replace('x','') #remove x in text
  allArticles[i] = ' '.join(word for word in allArticles[i].split() if word not in STOPWORDS) # remove stop words

In [47]:
allArticles[0]

'us interest rate rise epected us interest rates epected rise fifth time since june following us federal reserve latest rate setting meeting later tuesday borrowing costs tipped rise quarter percentage point move comes recovery us economy world biggest shows signs robustness sustainability dollar record breaking decline meanwhile spooked markets along high oil prices raised concerns pace inflation seeing evidence inflation moving higher said ken kim analyst stone mccarthy research risk actually happening mr kim added borrowing costs could rise fed said move measured way combat price growth lift interest rates year lows prompted sluggish us global growth economic picture looking rosy fed implemented quarter percentage point rises june august september november although us economy grew annual rate three months september analysts warn fed careful move aggressively take wind recovery sails earlier month figures showed job creation still weak consumer confidence subdued think fed feels fair

In [48]:
#df with word2vec
cols = ['article', 'labels']
d = {'article':allArticles, 'labels':allClsIds}
df = pd.DataFrame(data=d, columns=cols)
df.head()

,article,labels
0,us interest rate rise epected us interest rate...,0
1,uk economy facing major risks uk manufacturing...,0
2,ask jeeves tips online ad revival ask jeeves b...,0
3,us company admits benin bribery us defence tel...,0
4,ticking budget facing us budget proposals laid...,0


In [49]:
#TF-IDF:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['article'])
print(vectorizer.get_feature_names_out())
print(X.shape)
X

['aa' 'aaa' 'aaas' ... 'zutons' 'zvonareva' 'zvyagintsev']
(2225, 27675)


<2225x27675 sparse matrix of type '<class 'numpy.float64'>'
	with 339218 stored elements in Compressed Sparse Row format>

In [50]:
y = df['labels']


In [51]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=36)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(1780, 27675) (445, 27675) (1780,) (445,)


In [52]:
dt = DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors=3)
lc = SGDClassifier()
lr = LogisticRegression()
svc = SVC(kernel='linear')

models = [("DT",dt), ("KNN", knn), ("LC", lc), ("LR", lr), ("SVC", svc)]

In [53]:
from sklearn.preprocessing import StandardScaler
for mn, model in models:
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(mn,":",accuracy_score(y_test, y_pred))

DT : 0.8449438202247191
KNN : 0.9370786516853933
LC : 0.9797752808988764
LR : 0.9752808988764045
SVC : 0.9820224719101124


In [54]:
import joblib

In [ ]:
filename = '/content/2A_BBC_TF-IDF_PreTrained_Models/2A_BBC_TF_IDF_Logistic_Regression.sav'
joblib.dump(lr, filename)

['/content/2A_BBC_PreTrained_Models/2A_BBC_Logistic_Regression.sav']

In [ ]:
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)
print(result)

0.9752808988764045


In [ ]:
dt_file = '/content/2A_BBC_TF-IDF_PreTrained_Models/2A_BBC_TF_IDF_DecisionTreeClassifier.sav'
knn_file = '/content/2A_BBC_TF-IDF_PreTrained_Models/2A_BBC_TF_IDF_KNeighborsClassifier.sav'
lc_file = '/content/2A_BBC_TF-IDF_PreTrained_Models/2A_BBC_TF_IDF_SGDClassifier.sav'
lr_file = '/content/2A_BBC_TF-IDF_PreTrained_Models/2A_BBC_TF_IDF_Logistic_Regression.sav'
svc_file = '/content/2A_BBC_TF-IDF_PreTrained_Models/2A_BBC_TF_IDF_SVC.sav'

dt_pt = joblib.load(dt_file)
knn_pt = joblib.load(knn_file)
lc_pt = joblib.load(lc_file)
lr_pt = joblib.load(lr_file)
svc_pt = joblib.load(svc_file)

models_pt = [("DT",dt_pt), ("KNN", knn_pt), ("LC", lc_pt), ("LR", lr_pt), ("SVC", svc_pt)]
for mn, model in models:
  acc = model.score(X_test, y_test)
  print(mn,":",acc)


DT : 0.8382022471910112
KNN : 0.9370786516853933
LC : 0.9820224719101124
LR : 0.9752808988764045
SVC : 0.9820224719101124


In [55]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/2A_BBC_TFIDF_Tained_Model/"
models_1 = [("TFIDF_DecisionTreeClassifier",dt), ("TFIDF_KNeighborsClassifier", knn), ("TFIDF_SGDClassifier", lc), ("TFIDF_LogisticRegression", lr), ("TFIDF_SVC", svc)]
for mn_1, model_1 in models_1:
  fname = folder_path+mn_1+".sav"
  joblib.dump(model_1, fname)

In [56]:
dt_pt = joblib.load(folder_path+"TFIDF_DecisionTreeClassifier.sav")
knn_pt = joblib.load(folder_path+"TFIDF_KNeighborsClassifier.sav")
lc_pt = joblib.load(folder_path+"TFIDF_SGDClassifier.sav")
lr_pt = joblib.load(folder_path+"TFIDF_LogisticRegression.sav")
svc_pt = joblib.load(folder_path+"TFIDF_SVC.sav")

models_pt = [("DT",dt_pt), ("KNN", knn_pt), ("LC", lc_pt), ("LR", lr_pt), ("SVC", svc_pt)]
for mn, model in models:
  acc = model.score(X_test, y_test)
  print(mn,":",acc)

DT : 0.8449438202247191
KNN : 0.9370786516853933
LC : 0.9797752808988764
LR : 0.9752808988764045
SVC : 0.9820224719101124
